# Integrate own Combiner
As mentioned in the previous section, to make this as easy to use as possible, this notebook provide a template for integrating own Combiner in the pipeline.

In [ ]:
from flask import Flask, request, Response, jsonify
import json
import spacy

app = Flask(__name__)

print("Loading Custom Combiner")


You can modify the process method, to make it combine the passed documents to a single document.
in the default case, this method, returns the first document.


In [ ]:
# returns the first document
def process(documents):

    return documents[0]['document']

In [ ]:
@app.route('/', methods=['get', 'post'])
def index():
    print("Incoming request:")
    req = json.loads(request.data)
    documents = req['documents']

    document = process(documents)

    return jsonify(
            {'document' : document,
            'pipelineConfig' : req['pipelineConfig'],
            'componentId' : req['componentId']}
            )


In [ ]:
class LoggingMiddleware(object):
    def __init__(self, app):
        self._app = app

    def __call__(self, env, resp):
        errorlog = env['wsgi.errors']
        #pprint.pprint(('REQUEST', env), stream=errorlog)

        def log_response(status, headers, *args):
            #pprint.pprint(('RESPONSE', status, headers), stream=errorlog)
            return resp(status, headers, *args)

        return self._app(env, log_response)


# Run at flask startup (https://stackoverflow.com/a/55573732)
with app.app_context():
    pass

You can change the port of the web service here, after running the following code the webservice will be available.

**NOTE**: It only accepts get and post requests, trying to open the web page will result in an error with code 500.

In [ ]:
if __name__ == '__main__':
    # MAKE SURE YOU DON'T USE THE SAME PORT ON THE OTHER PIPELINE COMPONENTS
    port = 5005
    print("Running app... on port: ", port)
    app.wsgi_app = LoggingMiddleware(app.wsgi_app)
    app.run(host='0.0.0.0', port=port)